In [ ]:
import pandas as pd

data = pd.read_csv('games.csv', index_col=0)

In [ ]:
# Cleaning / getting data ready for machine learning
data['venue'] = data['venue'].map({'Home' : 1, 'Away' : 0})     # convert venue to 1's and 0's
data['Date'] = pd.to_datetime(data['Date'])
data['opponent'] = data['Opponent'].astype('category').cat.codes # Converting opponent to integers
data = data.drop(columns=['Time'], inplace=False)                # Dropping time column
data = data[data["Team"] != "Arizona Coyotes"]                   # Team doesnt exist anymore

predict = ['venue']

In [ ]:
data

In [ ]:
# Function to compute rolling averages over the last 3 games. Games for which there isnt enough data (i.e. the first 3 games of each teams) are dropped

def rolling_averages(team, cols, new_cols, window=3):
    team = team.sort_values("Date")    # Getting team data organized chronologically
    rolling = team[cols].rolling(window, closed='left').mean()   # closed=left to ignore current row in sliding window
    team[new_cols] = rolling
    team = team.dropna(subset=new_cols) # dropping first rows because not enough data
    return team

In [ ]:
cols = ['G', 'GA', 'S', 'SV%', 'S%']   # wanted columns for rolling
new_cols = [f"{c}_rolling" for c in cols]
predictors = new_cols + predict                         # New predictors


games_data = data.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols, 3))   # Compute rolling averages
games_data = games_data.droplevel("Team")
games_data.index = range(games_data.shape[0])  # fixing index level
games_data

In [ ]:
games_data = games_data[
        (games_data['Team'] != 'Arizona Coyotes') &
        (games_data['Opponent'] != 'Arizona Coyotes')
        ]

In [ ]:
#####
# 1. Initialize Elo ratings
initial_elo = 1500
teams = games_data['Team'].unique()
elo_ratings = {team: initial_elo for team in teams}

elo_features = []

# 2. Loop through each game and update ratings
for idx, row in games_data.iterrows():
    team = row['Team']
    opponent = row['Opponent']
    venue = row['venue']
    result = row['Result']  # 1 if win, 0 if loss

    # Optional: home-ice advantage
    team_elo = elo_ratings[team] + (35 if venue == 'Home' else 0)
    opponent_elo = elo_ratings[opponent]

    # Store Elo features BEFORE the game
    elo_features.append({
        'team_elo': team_elo,
        'opponent_elo': opponent_elo,
        'elo_diff': team_elo - opponent_elo
    })

    # Calculate expected outcome
    expected_win = 1 / (1 + 10 ** ((opponent_elo - team_elo) / 400))

    # Elo update (K-factor can be tuned)
    k = 30
    change = k * (result - expected_win)
    elo_ratings[team] += change
    elo_ratings[opponent] -= change

# Convert Elo features to DataFrame
elo_df = pd.DataFrame(elo_features)

# Merge with combined_team_view
games_data = pd.concat([games_data.reset_index(drop=True), elo_df], axis=1)

####
new = ['team_elo', 'opponent_elo', 'elo_diff']
predictors = predictors + new

In [ ]:
games_data

In [ ]:
games_data['days_since_last'] = games_data.groupby('Team')['Date'].diff().dt.days
games_data['days_since_last'].fillna(-1, inplace=True)

games_data['goal_diff'] = games_data['G'] - games_data['GA']

test = ['goal_diff']
new_col = ['goal_diff_rolling']

games_data = games_data.groupby('Team').apply(lambda x: rolling_averages(x, test, new_col, 5))
games_data = games_data.droplevel('Team')
games_data.index = range(len(games_data))

games_data

In [ ]:
new_pred = ['days_since_last']
predictors = predictors + new_pred
predictors

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

model = XGBClassifier(random_state=10, )    # base model

# Defining Time Series Split
TSS = TimeSeriesSplit(n_splits=5)

test_model = RandomForestClassifier(random_state=10)

lin = BaggingClassifier(LogisticRegression(random_state=10, solver='liblinear', penalty='l2', max_iter = 1000))


In [ ]:
from sklearn.metrics import precision_score

# Function to make predictions given the data, input features and chosen model

def make_predictions(data, predictors, model):
    train = data[data['Date'] < '2024-04-19']
    #train = train[train['Date'] > '2022-10-06']
    test = data[data['Date'] > '2024-04-19']
    model.fit(train[predictors], train['Result'])
    preds = model.predict(test[predictors])
    combined  = pd.DataFrame(dict(actual=test['Result'], prediction = preds), index=test.index)
    precision = precision_score(test['Result'], preds)
    return combined, precision

In [ ]:
# Defining search space for GridSearchCV
search_grid = {
    'n_estimators': [50, 75, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.02, 0.03],
    'reg_alpha': [1, 5, 10],
    'reg_lambda': [1, 5, 10]

}

alt_search_grid = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [3, 6, 9],
    'min_samples_split': [3, 5, 10]
}

lin_search_grid = {
    # Logistic Regression hyperparameters (base_estimator__)
    'estimator__C' : [0.5, 0.8, 1.0],
    'n_estimators': [10, 50, 100, 150, 200],
}

GS = GridSearchCV(
    estimator = test_model,
    param_grid = alt_search_grid,
    scoring = 'neg_log_loss',
    refit = True,
    cv = TSS,
    verbose= 4
)

training = games_data[games_data['Date'] < '2024-04-19']  # Training using 2021-2024 data
#training = training[training['Date'] > '2022-10-06']
testing = games_data[games_data['Date'] > '2024-04-19']   # Testing on most recent season (2024-2025)


In [ ]:
GS.fit(training[predictors], training['Result'])     # Training

In [ ]:
GS.best_score_

In [ ]:
new_model = GS.best_estimator_
new_model

In [ ]:
combined, precision = make_predictions(games_data, predictors, new_model)
precision

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, log_loss
predictions = new_model.predict(testing[predictors])

print(classification_report(testing['Result'], predictions))

In [ ]:
# Create DataFrame pairing features with their importances
importances = pd.DataFrame({
    'Feature': predictors,
    'Importance': new_model.feature_importances_
})

# Sort by importance
importances = importances.sort_values(by='Importance', ascending=False)

# Display top features
print(importances.head(10))


In [ ]:
combined

In [ ]:
combined = combined.merge(games_data[['Date', 'Team', 'Opponent', 'Result']], left_index=True, right_index=True)
combined

In [ ]:
final = combined.merge(combined, left_on=['Date', 'Team'], right_on=['Date', 'Opponent'])  # few games will drop due to rolling windows
final

In [ ]:
final[(final['prediction_x'] == 1) & (final['prediction_y'] == 0)]['actual_x'].value_counts()

In [ ]:
final[(final['prediction_x'] == 0) & (final['prediction_y'] == 1)]['actual_y'].value_counts()

In [ ]:
print("Overall accuracy turned out to be ~60%.")
print("However, when merging predictions for both games, we can see that the model is 63.3% accurate when both teams predictions match.")

In [ ]:
total = 560 + 390
total

In [ ]:
560/total